In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [15]:
def scrape_one_new(html, url):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('span', class_='contentTitle').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url,
        'datePublished':date,
        'dateModified':date_m
    }

# for old web normal
def make_date_normal_old(hmd,time):
    year, month, day = hmd[:4], hmd[4:6], hmd[6:]
    hour, minute = time.split('時')
    minute = minute.strip('分')
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    return f"{year}-{month}-{day}T{hour}:{minute}"

def scrape_one_old(html, url):
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find('span', class_="contentTitle").text.strip()
    hmd_ = url.split('/')[-2]
    time_ = soup.find('span', id="news_time").text
    date = make_date_normal_old(hmd_, time_)
    genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB','ＮＨＫ','ＮＨＫニュース','']]
    article = soup.find(['div','p'], id="news_textbody").text
    if soup.find_all(['div','p'], id="news_textmore") != []:
        for textmore in soup.find_all(['div','p'], id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all(['div','p'], class_="news_add") != []:
        for newsadd in soup.find_all(['div','p'], class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':[],
        'url':url,
        'datePublished':date,
        'dateModified':""
    }

def get_archiveurl_from_id(ID, date):
    url1 = f'https://web.archive.org/web/*/https://www3.nhk.or.jp/news/html/{date}/k{ID}1000.html'
    url2 = f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/{date}/k{ID}1000.html'
    
    driver.get(url1)
    time.sleep(3)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(times between|1 time|times).*?<a href="(.+?)">', html)
    archiveurl = 'https://web.archive.org' + snap.group(2)
    
    if 'nhk' not in archiveurl:
        driver.get(url2)
        time.sleep(3)
        html = str(driver.page_source.encode('utf-8'))
        snap = re.search(r'(times between|1 time|times).*?<a href="(.+?)">', html)
        archiveurl = 'https://web.archive.org' + snap.group(2)
    return None if 'nhk' not in archiveurl else archiveurl

def get_article_from_archiveurl(archiveurl):
    response = requests.get(archiveurl)
    time.sleep(2)
    html = response.text
    url_true = 'http' + archiveurl.split('/http')[-1]
    if 'This page is not available on the web' in html:
        return None
    try:
        try:
            dic = scrape_one_new(html, url_true)
        except:
            dic = scrape_one_old(html, url_true)
        return dic
    except:
        return None

def js(dic, year):
    if dic == None:
        return
    with open(f'nhkweb{year}.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open(f'nhkweb{year}.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

# check ID

In [10]:
data = pd.read_json('nhkweb2019.json')
data.head()

,id,title,article,genre,keywords,url,datePublished,dateModified
0,k10011760881000,僕、迷惑ですか？,東日本大震災のあと、「僕」は売れっ子になりました。売り上げは何倍にも跳ね上がり、カジュアルだ...,[暮らし],[],https://www3.nhk.or.jp/news/html/20190104/k100...,2019-01-04T12:32:01+09:00,2019-01-04T12:32:02+09:00
1,k10011764791000,日米欧もアジアも 2018年 株価下落 景気先行きに不安根強く,ニューヨーク株式市場は31日、2018年最後の取り引きが行われ、ダウ平均株価は値上がりして取...,[ビジネス],[株価・為替],http://www3.nhk.or.jp/news/html/20190101/k1001...,2019-01-01T08:55,
2,k10011764801000,ロシア スパイ容疑で米国人の男拘束 米ロ関係一層悪化へ,ロシアの治安当局は、スパイ活動をしていた疑いでアメリカ人の男をモスクワ市内で拘束しました。米...,[国際],[],https://www3.nhk.or.jp/news/html/20190101/k100...,2019-01-01T08:39,
3,k10011764811000,独 メルケル首相 米欧の自国第一主義に懸念 新年前に演説,ドイツのメルケル首相は新年を前にした恒例のテレビ演説で「国際協調という確かなものが圧力にさら...,[国際],[],http://www3.nhk.or.jp/news/html/20190101/k1001...,2019-01-01T09:02,
4,k10011764821000,マーライオンも摩天楼も 新年祝う花火が照らす,日本より１時間遅れで新年を迎えたシンガポールでは、観光名所として知られるマーライオンの像の前...,[国際],[国際５],http://www3.nhk.or.jp/news/html/20190101/k1001...,2019-01-01T09:13,


# get missing url

In [14]:
data = pd.read_json('nhkweb2019.json')
data = data[['id', 'url']]
data.id = data.id.apply(lambda x:x[1:-4])
data['date'] = data.url.apply(lambda x:x.split('news/html/')[-1].split('/')[0])
data.iloc[150:160,:]

,id,url,date
150,1001176639,https://www3.nhk.or.jp/news/html/20190103/k100...,20190103
151,1001176640,http://www3.nhk.or.jp/news/html/20190103/k1001...,20190103
152,1001176641,http://www3.nhk.or.jp/news/html/20190103/k1001...,20190103
153,1001176642,https://www3.nhk.or.jp/news/html/20190103/k100...,20190103
154,1001176643,http://www3.nhk.or.jp/news/html/20190103/k1001...,20190103
155,1001176644,http://www3.nhk.or.jp/news/html/20190103/k1001...,20190103
156,1001176645,http://www3.nhk.or.jp/news/html/20190103/k1001...,20190103
157,1001176646,https://www3.nhk.or.jp/news/html/20190103/k100...,20190103
158,1001176647,https://www3.nhk.or.jp/news/html/20190103/k100...,20190103
159,1001176648,https://www3.nhk.or.jp/news/html/20190103/k100...,20190103


In [27]:
# test
get_archiveurl_from_id(1001176503, 20190101)

'https://web.archive.org/web/20190101072943/https://www3.nhk.or.jp/news/html/20190101/k10011765031000.html'

In [4]:
driver = webdriver.Chrome()

In [16]:
year = 2019

for i in range(160, 300):
    ID1, ID2 = data.iat[i,0], data.iat[i+1,0]
    date1, date2 = data.iat[i,2], data.iat[i+1,2]

    if int(ID1) + 1 == int(ID2): # continuous = no missing
        continue
    elif date1 == date2:  # not continuous, but in the same day
        print(ID1)
        for ID in range(int(ID1)+1, int(ID2)):
            archiveurl = get_archiveurl_from_id(ID, date1)
            if archiveurl != None:
                dic = get_article_from_archiveurl(archiveurl)
                js(dic, year)
    else:  # not continuous, not in the same day
        print(ID1)
        is_date1 = True
        for ID in range(int(ID1)+1, int(ID2)):
            if is_date1:
                archiveurl = get_archiveurl_from_id(ID, date1)
                if archiveurl:
                    dic = get_article_from_archiveurl(archiveurl)
                    js(dic, year)
                else:
                    archiveurl = get_archiveurl_from_id(ID, date2)
                    if archiveurl:
                        dic = get_article_from_archiveurl(archiveurl)
                        js(dic, year)
                        is_date1 = False
            else:
                archiveurl = get_archiveurl_from_ID(ID, date2)
                if archiveurl:
                        js(dic, year)

1001176660


In [ ]:
driver.page_source.encode("utf8")

# scrape additional data

In [22]:
# 2019
urls = pd.read_csv('link19.txt', header=None)
urls

,0
0,https://web.archive.org/web/20190101010957/htt...
1,https://web.archive.org/web/20190101014546/htt...
2,https://web.archive.org/web/20190101023520/htt...
3,https://web.archive.org/web/20190101034607/htt...
4,https://web.archive.org/web/20190101042642/htt...
...,...
178,https://web.archive.org/web/20190106112232/htt...
179,https://web.archive.org/web/20190106112234/htt...
180,https://web.archive.org/web/20190106135151/htt...
181,https://web.archive.org/web/20190106135200/htt...


In [40]:
# check ID already exist
data = pd.read_json('nhkweb2019.json')
ids = data.id.tolist()
urls = pd.read_csv('link19.txt', header=None)
urls = [url for url in urls[0] if url.split('/')[-1].split('.html')[0] not in ids]
print(len(urls))
urls[:10]

0


[]

In [26]:
year = 2019
ids = data['id']
for url in urls:

    response = requests.get(url)
    time.sleep(2)
    html = response.text
    url_true = 'http' + url.split('/http')[-1]
    if 'This page is not available on the web' in html:
        continue
    
    print(url)
    
    try:
        dic = scrape_one_new(html, url_true)
    except:
        dic = scrape_one_old(html, url_true)
    js(dic, year)

81000.html
{'id': 'k10011765681000', 'title': '「サンマ投げ」2000匹を人々が拾う 豊漁祈願 静岡', 'article': '一年の豊漁と漁の安全を願って集まった人たちにサンマを投げてふるまう新年恒例の行事が静岡県西伊豆町の港で行われました。\n\n\n\nこの行事は西伊豆町の安良里港で毎年１月２日の漁船の乗り初めに合わせて行われています。接岸された地元所属の大型サンマ漁船「第135豊幸丸」に乗組員15人が乗り込み、用意したおよそ2000匹のサンマを次々と港に投げていきました。サンマは豊幸丸が去年10月に岩手県沖400キロの太平洋で取り、２日のために冷凍保存していたものだということです。集まった人の中には持参したバケツや袋がいっぱいになるほど拾い集める姿も見られ、用意したサンマはおよそ10分ほどでなくなりました。参加した人たちは「毎年この行事を楽しみにしています。ことしもたくさん拾えたのでよい年になりそうです」とか、「サンマが手に当たって痛かったですが、早速焼いて食べます」などと話していました。', 'genre': ['暮らし', '地域'], 'keywords': [], 'url': 'https://www3.nhk.or.jp/news/html/20190102/k10011765681000.html', 'datePublished': '2019-01-02T14:26', 'dateModified': ''}
https://web.archive.org/web/20190102060117/https://www3.nhk.or.jp/news/html/20190102/k10011765571000.html
{'id': 'k10011765571000', 'title': '大みそかから並びました！豪華景品「仙台初売り」に長い列', 'article': '豪華な景品で知られる伝統の「仙台初売り」が行われ、朝早くから多くの人たちがお正月の風物詩を楽しみました。\n江戸時代から続く「仙台初売り」は、商人の心意気を示そうと豪華な景品をつけたことが始まりとされ、毎年１月２日に行われています。仙台市中心部ではこの伝統を守ろうとデパートや大型店なども初売りを２日に統一しています。老舗のお茶

KeyboardInterrupt: 

In [9]:
dic

{'id': 'k10011764801000',
 'title': 'ロシア スパイ容疑で米国人の男拘束 米ロ関係一層悪化へ',
 'article': 'ロシアの治安当局は、スパイ活動をしていた疑いでアメリカ人の男をモスクワ市内で拘束しました。米ロ関係が一段と悪化することになりそうです。\nロシア連邦保安庁は、スパイ活動をしていた疑いでアメリカ人のポール・ウィラン容疑者をモスクワ市内で拘束したと31日発表しました。具体的にどのような活動をしていたかなど詳しいことは明らかにしていません。去年７月には、アメリカのワシントンに住むロシア人の女がロシア政府の指示を受けてアメリカの政治団体に潜入しスパイ活動をした疑いでアメリカの司法当局に逮捕され、ロシア政府が強く反発していました。アメリカとロシアの関係は、５年前のクリミア併合や、去年11月、ウクライナ海軍の艦船がロシア側に銃撃、拿捕（だほ）された事件、それに2016年のアメリカ大統領選挙でのいわゆるロシア疑惑などを受け悪化の一途をたどっています。今回、ロシア側がアメリカ人をスパイ容疑で拘束したことで米ロ関係が一段と悪化することになりそうです。',
 'genre': ['国際'],
 'keywords': [],
 'url': 'https://www3.nhk.or.jp/news/html/20190101/k10011764801000.html',
 'datePublished': '2019-01-01T08:39',
 'dateModified': ''}